In [ ]:
import cv2

from ultralytics import solutions

# Open the video file
cap = cv2.VideoCapture("legacy/proof_of_concept/demo_data/deer.mp4")
assert cap.isOpened(), "Error reading video file"

# Get video properties: width, height, and frames per second (fps)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define points for a line or region of interest in the video frame
line_points = [(400, 100), (400, 620)]  # Line coordinates

# Initialize the video writer to save the output video
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

model = "models/yolo11n.pt"

# Initialize the Object Counter with visualization options and other parameters
counter = solutions.ObjectCounter(
    show=True,  							# Display the image during processing
    region=line_points,  					# Region of interest points
    model="models/best.pt",  			# Ultralytics YOLO11 model file
    line_width=2,  							# Thickness of the lines and bounding boxes
)

# Process video frames in a loop
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Use the Object Counter to count objects in the frame and get the annotated image
    im0 = counter.count(im0)

    # Write the annotated frame to the output video
    video_writer.write(im0)

# Release the video capture and writer objects
cap.release()
video_writer.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO  # pip install ultralytics

# =========================================
# 1) Geometry helpers: crossing detection
# =========================================
def orient(a, b, c):
    # 2D cross product (b-a) x (c-a)
    return (b[0]-a[0])*(c[1]-a[1]) - (b[1]-a[1])*(c[0]-a[0])

def segments_intersect(p1, p2, q1, q2):
    # Proper segment intersection (including touching)
    o1 = orient(p1, p2, q1)
    o2 = orient(p1, p2, q2)
    o3 = orient(q1, q2, p1)
    o4 = orient(q1, q2, p2)
    if (o1 == 0 and o2 == 0 and o3 == 0 and o4 == 0):
        # collinear: check bbox overlap
        return (min(p1[0], p2[0]) <= max(q1[0], q2[0]) and
                min(q1[0], q2[0]) <= max(p1[0], p2[0]) and
                min(p1[1], p2[1]) <= max(q1[1], q2[1]) and
                min(q1[1], q2[1]) <= max(p1[1], p2[1]))
    return (o1 == 0 or o2 == 0 or np.sign(o1) != np.sign(o2)) and \
           (o3 == 0 or o4 == 0 or np.sign(o3) != np.sign(o4))

def line_side(p, a, b):
    # +1 if p is to the left of a->b, -1 if right, 0 on the line
    s = orient(a, b, p)
    return 0 if s == 0 else (1 if s > 0 else -1)

# =========================================
# 2) Feature extraction stub (you customize)
# =========================================
def get_features(roi_bgr: np.ndarray) -> float:
    """
    Return a scalar feature X for the bbox image patch.
    Replace with YOUR features (color/texture CNN embedding, HOG, etc.)
    """
    if roi_bgr.size == 0:
        return 0.0
    gray = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2GRAY)
    mean_intensity = float(gray.mean())
    h, w = gray.shape[:2]
    area_norm = (h * w) / 1e4  # rough normalization
    return mean_intensity * np.log1p(area_norm)

# =======================================================
# 3) Tiny model: non-negative outputs via exp(w*x + b)
#    Optimized on grouped interval-sum squared loss
# =======================================================
class GroupedExpSGD:
    def __init__(self, lr=1e-3, epochs=300, l2=1e-6):
        self.w = 0.0
        self.b = 0.0
        self.lr = lr
        self.epochs = epochs
        self.l2 = l2

    def fit(self, X_events, intervals_event_ranges, dY):
        """
        X_events: (N_events,) scalar features for each crossing event, in time order
        intervals_event_ranges: list of (start_idx, end_idx) INCLUSIVE for event indices
                                covering each labeled interval in order
        dY: (K,) ground-truth increments for those intervals
        """
        X = np.asarray(X_events, float)
        K = len(intervals_event_ranges)
        assert K == len(dY)
        for _ in range(self.epochs):
            grad_w = self.l2 * self.w
            grad_b = self.l2 * self.b
            for k, (a, b) in enumerate(intervals_event_ranges):
                if a > b:  # no events in this interval
                    # Can't fit; leave residual for diagnostics
                    continue
                xk = X[a:b+1]
                rk = np.exp(self.w * xk + self.b)   # event outputs (>=0)
                yhat = rk.sum()
                err = (yhat - dY[k])                # dL/dyhat = err
                grad_w += err * np.sum(rk * xk)     # chain rule
                grad_b += err * np.sum(rk)
            # Gradient step
            self.w -= self.lr * grad_w
            self.b -= self.lr * grad_b
        return self

    def predict_events(self, X_events):
        X = np.asarray(X_events, float)
        return np.exp(self.w * X + self.b)          # >= 0

# =========================================
# 4) Main: run tracking, collect events
# =========================================
def run_and_train(
    video_path="legacy/proof_of_concept/demo_data/deer.mp4",
    model_path="models/best.pt",
    csv_path="../.local_data/counts_xy_true_vs_yolo_series.csv",          # must contain: frame_idx, cum_count (optionally true_count)
    out_path="object_counting_output.avi",
    line_points=[(320, 560), (1820, 540)],
    draw=True
):
    # Read labels
    labels = pd.read_csv(csv_path)
    if "frame_index" not in labels or "true_count" not in labels:
        raise ValueError("CSV must have columns: frame_index, true_count (optionally true_count).")
    labels = labels.sort_values("frame_index").reset_index(drop=True)

    # Build label checkpoints and increments
    L_frames = labels["frame_index"].to_numpy().astype(int)
    cum = labels["true_count"].to_numpy().astype(float)
    # Prefer increments from cum to be robust; fallback to provided true_count if you like
    dY = np.diff(np.concatenate([[0.0], cum]))     # length K == number of checkpoints

    # Video IO
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    vw = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # Line setup
    A = tuple(map(int, line_points[0]))
    B = tuple(map(int, line_points[1]))

    # YOLO tracker
    yolo = YOLO(model_path)

    # Tracking memory: last center per track id
    last_center = {}   # id -> (x,y)

    # Collected events
    event_frames = []  # frame_idx for each crossing event
    event_X = []       # scalar feature X per event

    f_idx = -1
    label_ptr = 0  # for overlaying cum truth if desired
    cum_pred_running = 0.0  # overlay of cumulative predicted events (optional)

    while True:
        ok, frame = cap.read()
        if not ok:
            break
        f_idx += 1

        # Run tracking on the current frame
        results = yolo.track(frame, persist=True, verbose=False)
        r = results[0]  # ultralytics Results
        boxes = getattr(r, "boxes", None)
        if boxes is not None and boxes.id is not None:
            ids = boxes.id.cpu().numpy().astype(int)
            xyxy = boxes.xyxy.cpu().numpy()  # (N,4)
            for tid, bb in zip(ids, xyxy):
                x1, y1, x2, y2 = bb.astype(int)
                cx = (x1 + x2) // 2
                cy = (y1 + y2) // 2
                curr = (int(cx), int(cy))
                prev = last_center.get(tid, None)

                # crossing test (center trajectory vs counting line)
                if prev is not None:
                    if segments_intersect(prev, curr, A, B) and (line_side(prev, A, B) * line_side(curr, A, B) <= 0):
                        # Crop ROI for features
                        xx1, yy1 = max(0, x1), max(0, y1)
                        xx2, yy2 = min(w, x2), min(h, y2)
                        roi = frame[yy1:yy2, xx1:xx2]
                        X = get_features(roi)
                        event_frames.append(f_idx)
                        event_X.append(float(X))

                        # (Optional) draw a flash on crossing
                        if draw:
                            cv2.circle(frame, curr, 8, (0, 255, 0), -1)
                last_center[tid] = curr

        # Draw line and current stats
        if draw:
            cv2.line(frame, A, B, (0, 0, 255), 2)

        vw.write(frame)

    cap.release()
    vw.release()
    cv2.destroyAllWindows()

    # ============================
    # Build intervals over EVENTS
    # ============================
    # Sort events by time (already in order because we appended by frame)
    event_frames = np.asarray(event_frames, int)
    event_X = np.asarray(event_X, float)
    N_events = len(event_X)

    # Map label checkpoints to contiguous event index ranges
    # For checkpoint k at frame L_frames[k], interval k is (prev_label_frame, current_label_frame]
    intervals = []
    start_event_idx = 0
    prev_f = -1
    for k, f_lab in enumerate(L_frames):
        # events with prev_f < frame_idx <= f_lab belong to this interval
        end_event_idx = start_event_idx - 1
        while end_event_idx + 1 < N_events and event_frames[end_event_idx + 1] <= f_lab:
            end_event_idx += 1
        intervals.append((start_event_idx, max(start_event_idx - 1, end_event_idx)))  # (a,b) inclusive; empty if a>b
        start_event_idx = end_event_idx + 1
        prev_f = f_lab

    # ============================
    # Train on grouped intervals
    # ============================
    model = GroupedExpSGD(lr=3e-3, epochs=400, l2=1e-6).fit(event_X, intervals, dY)

    # ============================
    # Evaluation (weak supervision)
    # ============================
    # 1) Interval-level error (how well we match increments)
    yhat_events = model.predict_events(event_X)  # per-event non-negative outputs
    inc_preds = []
    for a, b in intervals:
        inc_preds.append(0.0 if a > b else float(yhat_events[a:b+1].sum()))
    inc_preds = np.asarray(inc_preds)
    inc_true = dY

    # 2) Cumulative at checkpoints
    cum_pred = np.cumsum(inc_preds)
    cum_true = cum

    # Metrics
    def rmse(x, y): return float(np.sqrt(np.mean((np.asarray(x) - np.asarray(y))**2)))
    def mae(x, y):  return float(np.mean(np.abs(np.asarray(x) - np.asarray(y))))

    metrics = {
        "interval_RMSE": rmse(inc_preds, inc_true),
        "interval_MAE":  mae(inc_preds, inc_true),
        "cumulative_RMSE": rmse(cum_pred, cum_true),
        "cumulative_MAE":  mae(cum_pred, cum_true),
        "events_total": int(N_events),
        "intervals": len(intervals),
        "empty_intervals_with_positive_truth":
            int(sum(1 for (a,b),dy in zip(intervals, dY) if a > b and dy > 0))
    }

    print("=== Weak-supervision evaluation ===")
    for k, v in metrics.items():
        print(f"{k:>34}: {v}")

    # Example: get per-checkpoint details
    report = pd.DataFrame({
        "frame_idx": L_frames,
        "inc_true": inc_true,
        "inc_pred": inc_preds,
        "cum_true": cum_true,
        "cum_pred": cum_pred
    })
    print("\nCheckpoint report (head):\n", report.head())

    return model, (event_frames, event_X), report, metrics


if __name__ == "__main__":
    # Edit paths as needed; CSV columns required: frame_idx, cum_count (optionally true_count)
    run_and_train(
        video_path="legacy/proof_of_concept/demo_data/deer.mp4",
        model_path="models/best.pt",
        #csv_path="",
        out_path="object_counting_output.avi",
        line_points=[(320, 560), (1820, 540)],
        draw=True
    )
